# Superhero Name Generator

---

[Superhero Names Dataset](https://github.com/am1tyadav/superhero)

import data

In [ ]:
!git clone https://github.com/am1tyadav/superhero

In [ ]:
with open('superhero/superheroes.txt','r') as f:
  data=f.read()

data[:100]

In [ ]:
import tensorflow as tf
print(tf.__version__)

Create a tokenizer

In [4]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~',
    split='\n',
)

tokenizer.fit_on_texts(data)

char-to-index and index-to-char dictionaries

In [6]:
char_to_index=tokenizer.word_index
index_to_char = dict((v,k) for k,v in char_to_index.items())

converting between names and sequences

In [ ]:
names =  data.splitlines()
names[:10]

In [ ]:
tokenizer.texts_to_sequences(names[0])

In [10]:
def name_to_seq(name):
  return [tokenizer.texts_to_sequences(c)[0][0] for c in name]

In [ ]:
name_to_seq(names[0])

In [12]:
def seq_to_name(seq):
  return ''.join([index_to_char[i] for i in seq if i!=0])

In [ ]:
seq_to_name(name_to_seq(names[0]))

Creating sequences


In [15]:
sequences=[]

In [16]:
for name in names:
  seq=name_to_seq(name)
  if len(seq)>=2:
    sequences+=[seq[:i] for i in range (2,len(seq)+1)]

In [ ]:
sequences[:10]

In [ ]:
max_len=max([len(x) for x in sequences])
max_len

Padding all sequences

In [ ]:
paddded_sequences = tf.keras.preprocessing.sequence.pad_sequences(
    sequences,padding='pre',maxlen=max_len)

paddded_sequences[0]

In [ ]:
paddded_sequences.shape

Creating Training and Validation Sets



In [21]:
x,y=paddded_sequences[:,:-1],paddded_sequences[:,-1]

In [22]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y)



In [23]:
num_chars=len(char_to_index.keys())+1

In [ ]:
num_chars

Creating the Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPool1D, LSTM
from tensorflow.keras.layers import Bidirectional, Dense

model = Sequential([
                    Embedding(num_chars,8,input_length=max_len-1),
                    Conv1D(64,5,strides=1,activation='tanh',padding='causal'),
                    MaxPool1D(2),
                    LSTM(32),
                    Dense(num_chars,activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Training the Model

In [ ]:
h=model.fit(
    x_train,y_train,validation_data=(x_test,y_test),
    epochs=50,verbose=2,callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=2)]
)
    

In [ ]:
from matplotlib import pyplot as plt

epochs_ran = len(h.history['loss'])
plt.plot(range(0,epochs_ran),h.history['val_accuracy'],label='validation')
plt.plot(range(0,epochs_ran),h.history['accuracy'],label='Training')
plt.legend()
plt.show()

Generate Names!

In [28]:
def generate_names(seed):
  for i in range(0,40):
    seq=name_to_seq(seed)
    padded = tf.keras.preprocessing.sequence.pad_sequences(
        [seq],padding='pre',maxlen=max_len-1,truncating='pre')
    pred = model.predict(padded)[0]
    pred_char=index_to_char[tf.argmax(pred).numpy()]
    seed+=pred_char

    if pred_char == '\t':
      break
  print(seed)


In [ ]:
generate_names('am')